In [ ]:
class Block:
    def __init__(self, name):
        self.name = name
        self.on = None
        self.clear = True
        self.ontable = True
    
    def __repr__(self):
        return self.name


class BlocksWorldPlanner:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.goal_stack = []
        self.actions = []
        self.blocks = {}

    def run(self):
        self.parse_initial_state()
        self.goal_stack.append(self.goal_state)
        while self.goal_stack:
            goal = self.goal_stack[-1]
            if isinstance(goal, str):
                if goal.startswith('ON'):
                    self.handle_on_goal(goal)
                elif goal.startswith('ONTABLE'):
                    self.handle_ontable_goal(goal)
                elif goal.startswith('CLEAR'):
                    self.handle_clear_goal(goal)
            elif isinstance(goal, Block):
                self.handle_block_goal(goal)
        print("Actions:")
        for action in self.actions:
            print(action)

    def parse_initial_state(self):
        # Initialize Block objects
        for statement in self.initial_state.split(', '):
            block_name, position = statement.split()
            self.blocks[block_name] = Block(block_name)
            if position != "ontable":
                self.blocks[block_name].ontable = False
                self.blocks[block_name].clear = False
                self.blocks[block_name].on = self.blocks[position]
                self.blocks[position].clear = False

    def handle_on_goal(self, goal):
        # Handle the ON goal
        block1, block2 = goal[3:-1].split(',')
        if self.blocks[block1].on == self.blocks[block2]:
            self.goal_stack.pop()
        else:
            self.goal_stack.append(self.blocks[block1].on)

    def handle_ontable_goal(self, goal):
        # Handle the ONTABLE goal
        block = goal[8:]
        if self.blocks[block].ontable:
            self.goal_stack.pop()
        else:
            self.goal_stack.append(block)

    def handle_clear_goal(self, goal):
        # Handle the CLEAR goal
        block = goal[6:]
        if self.blocks[block].clear:
            self.goal_stack.pop()
        else:
            for b in self.blocks.values():
                if b.clear:
                    self.goal_stack.append(b)
                    break

    def handle_block_goal(self, goal):
        # Handle a block goal
        if goal.clear and goal.ontable:
            self.goal_stack.pop()
        elif goal.clear:
            self.goal_stack.append("ONTABLE({})".format(goal.name))
        elif goal.ontable:
            self.goal_stack.append("CLEAR({})".format(goal.name))
        else:
            self.goal_stack.append("CLEAR({})".format(goal.name))
            self.goal_stack.append("ONTABLE({})".format(goal.name))
        if goal.on:
            self.goal_stack.append("ON({}, {})".format(goal.on.name, goal.name))

    def add_action(self, action):
        # Add an action to the list of actions
        self.actions.append(action)

    def move(self, block, source, destination):
        # Perform a move action
        self.add_action("Move({}, {}, {})".format(block, source, destination))
        if source == 'ontable':
            self.blocks[block].ontable = True
            self.blocks[block].clear = True
        else:
            self.blocks[source].on = None
            self.blocks[source].clear = True
        if destination == 'ontable':
            self.blocks[block].ontable = True
            self.blocks[block].clear = False
        else:
            self.blocks[destination].on = self.blocks[block]
            self.blocks[destination].clear = False


# Initial state: A on B, C on D, E on table
initial_state = "A B, C D, E ontable"

# Goal state: A on D, B on table, C on table, E on A
goal_state = "A D, B ontable, C ontable, E A"

planner = BlocksWorldPlanner(initial_state, goal_state)
planner.run()
